# UE1: Calibration Curve
Inside the "linear_regression_[group number].csv" files are datasets of concentrations and corresponding absorption values from a UV/VIS 
determination of protein concentration. Using this dataset, carry out the following steps to establish a calibration: 

1. determine the linear range of the method
2. calculate the calibration curve
3. determine the LOD and LOQ of the calibration

In [ ]:
import pandas as pd


import matplotlib.pyplot as plt
import numpy as np
import xarray as xr


try:
    from cavs_style.style import CAVS_style

    plt.style.use(CAVS_style)
except ImportError:
    pass

%matplotlib inline

## 1. Linear Range

In [2]:
idx, concentration, absorption = np.genfromtxt("linear_regression_2.csv", delimiter=',', unpack=True)

ValueError: Some errors were detected !
    Line #2 (got 1 columns instead of 3)
    Line #3 (got 1 columns instead of 3)
    Line #4 (got 1 columns instead of 3)
    Line #5 (got 1 columns instead of 3)
    Line #6 (got 1 columns instead of 3)
    Line #7 (got 1 columns instead of 3)
    Line #8 (got 1 columns instead of 3)
    Line #9 (got 1 columns instead of 3)
    Line #10 (got 1 columns instead of 3)
    Line #11 (got 1 columns instead of 3)
    Line #12 (got 1 columns instead of 3)
    Line #13 (got 1 columns instead of 3)
    Line #14 (got 1 columns instead of 3)
    Line #15 (got 1 columns instead of 3)
    Line #16 (got 1 columns instead of 3)
    Line #17 (got 1 columns instead of 3)
    Line #18 (got 1 columns instead of 3)
    Line #19 (got 1 columns instead of 3)
    Line #20 (got 1 columns instead of 3)
    Line #21 (got 1 columns instead of 3)
    Line #22 (got 1 columns instead of 3)
    Line #23 (got 1 columns instead of 3)
    Line #24 (got 1 columns instead of 3)
  

In [ ]:
protain_da = xr.DataArray(absorption,
                          dims=['idx', 'c'],
                          coords={'c': ('c', concentration),
                                     'idx': idx})

protain_da

In [ ]:
protain_da.plot.line(x='c', add_legend=False)
plt.show()